# PEFT 进阶操作

## 1. 自定义模型适配

In [1]:
import torch
from torch import nn
from peft import LoraConfig, get_peft_model, PeftModel

In [2]:
net1 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net1

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [3]:
for name, param in net1.named_parameters():
    print(name)

0.weight
0.bias
2.weight
2.bias


In [4]:
config = LoraConfig(target_modules=["0"])

In [5]:
model1 = get_peft_model(net1, config)

In [6]:
model1

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): Linear(
        in_features=10, out_features=10, bias=True
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

## 2. 多适配器加载与切换

In [7]:
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net2

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [8]:
config1 = LoraConfig(target_modules=["0"])
model2 = get_peft_model(net2, config1)
model2.save_pretrained("./loraA")

In [9]:
config2 = LoraConfig(target_modules=["2"])
model2 = get_peft_model(net2, config2)
model2.save_pretrained("./loraB")

In [10]:
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net2

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [11]:
model2 = PeftModel.from_pretrained(net2, model_id="./loraA/", adapter_name="loraA")
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): Linear(
        in_features=10, out_features=10, bias=True
        (lora_dropout): ModuleDict(
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

In [12]:
model2.load_adapter("./loraB/", adapter_name="loraB")
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): Linear(
        in_features=10, out_features=10, bias=True
        (lora_dropout): ModuleDict(
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
      )
      (1): ReLU()
      (2): Linear(
        in_features=10, out_features=2, bias=True
        (lora_dropout): ModuleDict(
          (loraB): Identity()
        )
        (lora_A): ModuleDict(
          (loraB): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (loraB): Linear(in_features=8, out_features=2, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
     

In [13]:
model2.active_adapter

'loraA'

In [14]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[-1.0515, -1.3579]])

In [15]:
for name, param in model2.named_parameters():
    print(name, param)

base_model.model.0.weight Parameter containing:
tensor([[ 0.0986, -0.0116,  0.0064,  0.0869,  0.2262, -0.3124,  0.1880,  0.3149,
          0.2943, -0.0315],
        [-0.0699, -0.0435,  0.0111, -0.1133,  0.3130,  0.0972, -0.2215,  0.1551,
          0.2324,  0.0888],
        [ 0.2493,  0.1299,  0.2266, -0.1462,  0.1131,  0.3011,  0.1412,  0.1223,
          0.2984,  0.1015],
        [ 0.1902,  0.0493, -0.0610,  0.0432, -0.2254, -0.2858,  0.1830, -0.0072,
         -0.3141, -0.1154],
        [-0.1394,  0.2665,  0.1222, -0.2287, -0.1341, -0.0824, -0.2547, -0.0844,
          0.2754,  0.2924],
        [-0.1255,  0.1472, -0.1192,  0.1369, -0.2551, -0.0734,  0.0825,  0.2721,
          0.1332,  0.2202],
        [ 0.2424,  0.0797, -0.0516,  0.2836, -0.0615, -0.0863,  0.1314,  0.2934,
         -0.2413,  0.1712],
        [ 0.1404, -0.1963, -0.2448, -0.1210,  0.0751,  0.1120, -0.0822,  0.1745,
          0.0505,  0.2288],
        [-0.2516, -0.2477, -0.0183, -0.1723, -0.1651,  0.2466, -0.0038,  0.0557,

In [16]:
for name, param in model2.named_parameters():
    if name in ["base_model.model.0.lora_A.loraA.weight", "base_model.model.0.lora_B.loraA.weight"]:
        param.data = torch.ones_like(param)

In [17]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[ 92.0650, -21.8470]])

In [18]:
model2.set_adapter("loraB")

In [19]:
model2.active_adapter

'loraB'

In [20]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[-1.0515, -1.3579]])

## 3. 禁用适配器

In [21]:
model2.set_adapter("loraA")

In [22]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[ 92.0650, -21.8470]])

In [23]:
with model2.disable_adapter():
    print(model2(torch.arange(0, 10).view(1, 10).float()))

tensor([[-1.0515, -1.3579]])
